# Tech Challenge

This is the ETL discovery and implementation to retrieve the data from:
https://datos.cdmx.gob.mx/dataset/prueba_fetchdata_metrobus/resource/ad360a0e-b42f-482c-af12-1fd72140032e?inner_span=True 



In [1]:
#Import base libraries and try a connection to database
import json
import requests
#ORM and GeoORM libraries
from sqlalchemy import create_engine, Column, Integer, String, Float,DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import shapely.wkt
from geoalchemy2 import Geometry
from geoalchemy2.shape import to_shape, from_shape
from shapely.geometry import Polygon, Point,shape

In [2]:
#Database URLS
base_url = "postgresql://{}:{}@{}:{}/{}"
#Docker URL
docker_url = base_url.format("docker","docker","localhost","54321","gis")
#Local URL
local_url =  base_url.format("postgres","TaekoBB","localhost","5432","stations")

In [3]:
#Auxiliar function to retrieve only the values from the dictionaries to simply the constructor input value
def transform_to_list(register):
    register_list = []
    register_values = register.values()
    register_list = list(register_values)
    return register_list

In [4]:
#Start the database.
engine = create_engine(docker_url)

In [5]:
#Retrieve the base model to generate the entities
Base = declarative_base()

In [6]:
#Generate the database session
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
#Declare the model Mayoralty
class Mayoralty(Base):
    __tablename__ = 'mayoralties'
    
    id = Column(Integer, primary_key=True)
    nomgeo = Column(String)
    cve_mun = Column(Integer)
    cve_ent = Column(Integer)
    cvegeo = Column(Integer)
    geo_point_2d = Column(String)
    geo_shape = Column(Geometry("POLYGON", srid=4326, spatial_index=True))
    municipio = Column(Integer)
    
    def __init__(self,row):
        geom_json = json.loads(row[7])
        valid_geom = shape(geom_json)
        self.id = row[0]
        self.nomgeo = row[2]
        self.cve_mun = row[3]
        self.cve_ent = row[4]
        self.cvegeo = row[5]
        self.geo_point_2d = row[6]
        self.geo_shape = from_shape(valid_geom)
        self.municipio = row[8]

In [8]:
#Declare the model Transport Unit
class TransportUnit(Base):
    __tablename__ = 'transport_units'
    
    id = Column(Integer, primary_key = True)
    date_updated = Column(DateTime)
    vehicle_id = Column(Integer)
    vehicle_label = Column(Integer)
    vehicle_current_status = Column(Integer)
    position_latitude = Column(Float)
    position_longitude = Column(Float)
    geographic_point = Column(Geometry("POINT", srid = 4326, spatial_index = True))
    position_speed = Column(Integer)
    position_odometer = Column(Integer)
    trip_schedule_relationship = Column(Integer)
    trip_id = Column(Integer, nullable = True)
    trip_start_date = Column(Integer, nullable = True)
    trip_route_id = Column(Integer, nullable = True)
    mayoralty = Column(String, nullable = True)
    
    def __init__(self,row):
        self.id = row[0]
        self.date_updated = row[2]
        self.vehicle_id = row[3]
        self.vehicle_label = row[4]
        self.vehicle_current_status = row[5]
        self.position_latitude = float(row[6])
        self.position_longitude = float(row[7])
        self.geographic_point = from_shape(Point(self.position_longitude, self.position_latitude), srid=4326)
        self.position_speed = row[9]
        self.position_odometer = row[10]
        self.trip_schedule_relationship = row[11]
        self.trip_id = row[12] 
        self.trip_start_date = row[13]
        self.trip_route_id = row[14]

In [9]:
#MOCK data for the 
#record = {
#                "_id": 1,
#                "id": 0,
#                "nomgeo": "Milpa Alta",
#                "cve_mun": 9,
#                "cve_ent": 9,
#                "cvegeo": 9009,
#                "geo_point_2d": "19.1394565999,-99.0510954218",
#                "geo_shape": "{\"type\": \"Polygon\", \"coordinates\": [[[-98.99723373999994, 19.22731068999911], [-98.99744704999993, 19.227184459999123], [-98.9974635999999, 19.226488459999103], [-98.99706316999995, 19.226423519999155], [-98.99668105999993, 19.226326379999104], [-98.99642859999992, 19.22619039999913], [-98.99638268999992, 19.226128889999114], [-98.9962694499999, 19.225977169999148], [-98.9961805999999, 19.2258887699991], [-98.996362, 19.225260769999117], [-98.99637625999996, 19.225211429999153], [-98.99611645999993, 19.22513468999913], [-98.99579087999992, 19.225038489999125], [-98.99544217999993, 19.226242349999136], [-98.99542281999994, 19.22643461999916], [-98.9952639099999, 19.227514919999074], [-98.99439442999991, 19.22744791999911], [-98.99420421999996, 19.227426029999116], [-98.99372541999993, 19.227470459999125], [-98.99366731999997, 19.227278229999108], [-98.9936106599999, 19.227090769999162], [-98.9935620799999, 19.22693002999914], [-98.99354201999991, 19.22679701999911], [-98.99349722999993, 19.22649993999914], [-98.9934591099999, 19.22624707999914], [-98.99339530999997, 19.225823889999145], [-98.9932627799999, 19.224944849999105], [-98.9929066499999, 19.224918479999122], [-98.99255073999993, 19.224892109999107], [-98.99249385999995, 19.224887879999127], [-98.991868, 19.22484133999917], [-98.99123725999996, 19.224789769999113], [-98.99107357999993, 19.22559467999912], [-98.9905586799999, 19.225778349999146], [-98.9882339099999, 19.223704769999124], [-98.98571903, 19.22143465999913], [-98.98437096999994, 19.220300849999123], [-98.98431784999993, 19.220256169999093], [-98.98239261999994, 19.218637139999164], [-98.9781500299999, 19.21506830999912], [-98.97789667999992, 19.21508231999905], [-98.97501030999994, 19.214591199999116], [-98.97495757999994, 19.214585349999144], [-98.97450208999997, 19.214483199999172], [-98.97434953999995, 19.21444313999915], [-98.97371659999995, 19.21412779999909], [-98.97362139999993, 19.21408291999914], [-98.9733322599999, 19.213822489999085], [-98.97321267999993, 19.213664509999166], [-98.97317684999992, 19.21361556999913], [-98.97317681999995, 19.213615509999155], [-98.9730590299999, 19.213454459999134], [-98.97282770999995, 19.213170539999187], [-98.97242856999993, 19.21269541999913], [-98.97222425999995, 19.212510909999125], [-98.97197039999993, 19.212326479999135], [-98.97170613999994, 19.21213022999912], [-98.9714977799999, 19.211971549999106], [-98.97129551999994, 19.211864629999088], [-98.97101601999994, 19.211716079999107], [-98.96997776999993, 19.21127537999915], [-98.9699306799999, 19.21125521999914], [-98.9697610599999, 19.211182569999128], [-98.96947313999996, 19.210974179999102], [-98.96924021999992, 19.210850829999085], [-98.96892605999992, 19.210712449999185], [-98.96857757999994, 19.210589059999144], [-98.96852791999994, 19.210571079999085], [-98.96837654999995, 19.21048884999913], [-98.96765733999992, 19.209796049999113], [-98.96683774999994, 19.209006549999106], [-98.96682650999995, 19.208961339999124], [-98.96686347999994, 19.208742679999112], [-98.96700601999994, 19.207899429999127], [-98.96705179999994, 19.20762859999915], [-98.96742396999993, 19.207327199999153], [-98.96718134999995, 19.207042249999105], [-98.96716850999995, 19.206617599999156], [-98.96722873999992, 19.20625108999912], [-98.96740454999993, 19.206172429999143], [-98.9675094199999, 19.20612548999908], [-98.96797916999996, 19.205719979999092], [-98.96857196999993, 19.20520822999913], [-98.96967856999997, 19.204252909999138], [-98.9677018899999, 19.203128849999125], [-98.96768027999993, 19.203025919999153], [-98.96760965999992, 19.202689739999137], [-98.9670258299999, 19.199909859999114], [-98.9669786899999, 19.197921649999156], [-98.96769388999992, 19.189743079999086], [-98.96641033999997, 19.18703165999913], [-98.95984905999994, 19.17916787999908], [-98.95652173999994, 19.17663521999913], [-98.9561746799999, 19.176763059999168], [-98.95491573999995, 19.174063399999092], [-98.9523856499999, 19.16863761999907], [-98.96561216999997, 19.164930459999145], [-98.9673663999999, 19.164438689999127], [-98.9652502599999, 19.161961689999135], [-98.9646160299999, 19.16121928999915], [-98.96373816999991, 19.160191679999105], [-98.96312268999996, 19.159471199999125], [-98.96269744999994, 19.158973399999105], [-98.96201925999993, 19.158179509999098], [-98.96108434999994, 19.1570850599991], [-98.9605706199999, 19.156483629999112], [-98.96001744999994, 19.155836059999142], [-98.9598605199999, 19.155652339999175], [-98.95946831999994, 19.155193199999143], [-98.9584844299999, 19.154041339999115], [-98.9583185399999, 19.15384711999913], [-98.95462630999997, 19.14952434999912], [-98.9594818899999, 19.147298419999093], [-98.95989550999992, 19.14710879999916], [-98.96260419999996, 19.145866949999107], [-98.96303702999994, 19.14564259999914], [-98.96645593999993, 19.143870509999118], [-98.96595605999993, 19.142584489999155], [-98.96566316999996, 19.141830969999152], [-98.9654936599999, 19.141394909999125], [-98.96511271999996, 19.14041481999914], [-98.96508905999993, 19.14038599999913], [-98.96464529, 19.139845419999094], [-98.96460554999997, 19.13979701999916], [-98.9638887199999, 19.13821747999912], [-98.96376030999993, 19.137934519999106], [-98.96348754999995, 19.137488079999116], [-98.96317541999993, 19.136977139999146], [-98.9628977099999, 19.136522549999118], [-98.96262621999992, 19.136078149999122], [-98.96247710999992, 19.13583407999912], [-98.9623770199999, 19.135670249999134], [-98.96227267999994, 19.13545579999912], [-98.96194547999991, 19.13478334999917], [-98.96181716999992, 19.134519689999085], [-98.9617153999999, 19.134310539999124], [-98.96162554999991, 19.134125889999137], [-98.96147839999993, 19.133823449999106], [-98.96109188999993, 19.133029109999118], [-98.96085396999992, 19.132305229999123], [-98.96050981999996, 19.13125811999913], [-98.96043245999991, 19.131022769999163], [-98.9595422199999, 19.128314109999145], [-98.95872948999993, 19.125841229999136], [-98.95852896999995, 19.125273799999132], [-98.95779737999997, 19.123203579999124], [-98.95750747999993, 19.12238321999919], [-98.9576560299999, 19.11963101999909], [-98.95800284999991, 19.117915349999134], [-98.95807042999994, 19.117580969999153], [-98.95850024999994, 19.114897939999114], [-98.95856270999994, 19.11450804999909], [-98.958702, 19.113638509999127], [-98.95869201999992, 19.11270524999909], [-98.95869054999994, 19.11256793999914], [-98.95840974999997, 19.11130939999913], [-98.95860637999995, 19.110413879999143], [-98.95867647999994, 19.11009461999912], [-98.95895665999991, 19.109042719999117], [-98.95906351, 19.104277949999112], [-98.96123977999993, 19.095583179999135], [-98.9632657699999, 19.09206277999915], [-98.96572147999994, 19.085861169999127], [-98.96828357999995, 19.08314087999909], [-98.97137233999995, 19.081218379999136], [-98.97369661999993, 19.07955316999914], [-98.97899539999993, 19.074455949999127], [-98.98236136999994, 19.077608419999134], [-98.9835944899999, 19.078763249999145], [-99.02899367999994, 19.085368799999134], [-99.03019490999993, 19.084556539999102], [-99.0432088599999, 19.075755459999133], [-99.0487286599999, 19.066601539999088], [-99.05665036999993, 19.053462029999135], [-99.05980022999991, 19.048236659999116], [-99.06208574999991, 19.04871871999915], [-99.1044538599999, 19.05764802999912], [-99.10512856999995, 19.057790119999105], [-99.12543311999995, 19.062064719999103], [-99.13521261999993, 19.088840549999155], [-99.13537901999995, 19.089296079999176], [-99.13648147999994, 19.089377979999156], [-99.14681085999992, 19.100583089999105], [-99.15474753999993, 19.10883654999911], [-99.1632928499999, 19.117490569999116], [-99.15500931999996, 19.135309709999106], [-99.15400427999997, 19.13812156999914], [-99.1524995799999, 19.14211807999914], [-99.15195690999992, 19.150767429999107], [-99.1518057699999, 19.151410749999137], [-99.14972374999992, 19.159844089999098], [-99.1496084199999, 19.160312879999147], [-99.14870414999994, 19.160273029999118], [-99.14820007999994, 19.160235029999082], [-99.14757857999994, 19.160140629999116], [-99.1472081699999, 19.160041509999086], [-99.1468038899999, 19.15994468999909], [-99.1465098199999, 19.159811619999108], [-99.1460768599999, 19.159552139999143], [-99.14545782999991, 19.159213719999112], [-99.14492217999994, 19.15892833999912], [-99.14480454999993, 19.15886402999912], [-99.14449414999991, 19.158665449999127], [-99.1442300199999, 19.158399049999158], [-99.14396081999992, 19.158116849999114], [-99.14367707999993, 19.157834649999128], [-99.1434458199999, 19.157538849999103], [-99.14323824999992, 19.1571432899991], [-99.14305619999998, 19.15681850999915], [-99.14293831999996, 19.156513969999125], [-99.1427495499999, 19.15608034999908], [-99.1425192299999, 19.15555759999913], [-99.14228364999994, 19.155136379999146], [-99.14203407999995, 19.154743319999092], [-99.1418225199999, 19.154480059999102], [-99.14152397999995, 19.154059719999143], [-99.14113294999994, 19.153514549999137], [-99.14084908999995, 19.152918399999127], [-99.14069556999996, 19.15249668999916], [-99.14040431999992, 19.1522132599991], [-99.14002019999994, 19.15195856999911], [-99.13977424999992, 19.151807369999112], [-99.13942614999995, 19.151647219999123], [-99.13894864999995, 19.151443449999125], [-99.13853225999993, 19.151338349999133], [-99.13777331999991, 19.150995619999133], [-99.1372113499999, 19.15075037999913], [-99.13240604999994, 19.148611289999163], [-99.13123153999997, 19.15075037999913], [-99.13024431999993, 19.152544169999107], [-99.11913013999997, 19.172872949999128], [-99.11865413999995, 19.17374265999913], [-99.11341834999993, 19.183302289999098], [-99.11266061999997, 19.18469265999916], [-99.11233062999995, 19.18529547999914], [-99.11166244999997, 19.18651328999915], [-99.11148408999993, 19.18683413999909], [-99.11027633999996, 19.18904448999916], [-99.10573639999991, 19.197340709999136], [-99.10527767999996, 19.19805801999917], [-99.1051818299999, 19.198273079999115], [-99.10469782999995, 19.198205519999114], [-99.10465834999994, 19.19827928999913], [-99.10463722999997, 19.198336799999165], [-99.10463147999995, 19.198382019999134], [-99.10460885999994, 19.198425819999155], [-99.10472545999993, 19.1984933499991], [-99.1047789199999, 19.198560919999128], [-99.10482430999994, 19.198656149999096], [-99.10485096999996, 19.19876628999909], [-99.10484422999993, 19.19886610999915], [-99.10479968999991, 19.19896421999914], [-99.10472557999996, 19.199046369999152], [-99.1041963499999, 19.19936717999908], [-99.1041233399999, 19.199410659999163], [-99.10408061999998, 19.199436379999163], [-99.10326791999992, 19.199926199999144], [-99.10311387999992, 19.200018289999154], [-99.10262924999991, 19.200311119999146], [-99.10239984999994, 19.20046476999912], [-99.1022731099999, 19.200628229999136], [-99.10224693999997, 19.200661969999164], [-99.10217033999994, 19.200827969999118], [-99.10211757999993, 19.201051799999124], [-99.10208644999993, 19.20157231999912], [-99.10207005999995, 19.2018463999991], [-99.10202242999993, 19.202280519999128], [-99.10196359999996, 19.202646089999092], [-99.1019331799999, 19.203434249999106], [-99.1018997099999, 19.203823419999143], [-99.10187084999991, 19.20420084999914], [-99.10185431, 19.204430429999157], [-99.10181474999992, 19.20462162999914], [-99.1017804299999, 19.204719179999113], [-99.1017459399999, 19.204808889999107], [-99.10169308999994, 19.204891779999098], [-99.1015900199999, 19.205014859999157], [-99.10100227999995, 19.205700799999132], [-99.10098441999993, 19.205721649999102], [-99.10088611999994, 19.205824649999126], [-99.100802, 19.205894979999098], [-99.10069887999995, 19.205955289999185], [-99.10060633999991, 19.205998029999144], [-99.10037511999995, 19.20607247999913], [-99.10025611999995, 19.206115229999128], [-99.10018208999995, 19.20617048999908], [-99.10012394999991, 19.206245849999142], [-99.10008430999991, 19.2063211799991], [-99.10005791999995, 19.20641408999914], [-99.1000447499999, 19.206519549999154], [-99.10002610999994, 19.206993349999127], [-99.10001296999995, 19.207143999999115], [-99.09997601999996, 19.207267049999096], [-99.09992053999996, 19.207385629999095], [-99.0998306599999, 19.207495599999138], [-99.09936913999994, 19.207990849999117], [-99.09930585999992, 19.208069419999127], [-99.09926941999994, 19.208105149999092], [-99.09921250999996, 19.208147279999114], [-99.09916321999994, 19.20816431999911], [-99.09910994999994, 19.208182819999124], [-99.0990373699999, 19.208196569999124], [-99.09894507999992, 19.208209399999145], [-99.09889554999991, 19.208211379999103], [-99.09881945999994, 19.20821501999917], [-99.09848844999998, 19.208215199999103], [-99.09841050999991, 19.208220939999148], [-99.0983506199999, 19.208226659999088], [-99.09826902999997, 19.20823819999912], [-99.09813737999993, 19.20827237999911], [-99.0979443099999, 19.20832996999912], [-99.09713865999994, 19.208592719999174], [-99.09699591999997, 19.208630749999106], [-99.09687310999995, 19.20865385999913], [-99.09677734999993, 19.208664449999155], [-99.09667973999991, 19.208664829999105], [-99.0965507399999, 19.208652079999087], [-99.09643277, 19.208626689999132], [-99.09627317999993, 19.2085987699991], [-99.09612293999994, 19.20857299999911], [-99.09597408999993, 19.20855413999912], [-99.09585611999994, 19.20855591999913], [-99.09577444999992, 19.20857834999911], [-99.09568702999995, 19.208627939999115], [-99.09561151999996, 19.208717889999125], [-99.09555505999994, 19.20880314999913], [-99.0952531499999, 19.209259029999167], [-99.0951595799999, 19.2094386499991], [-99.09509891999991, 19.20959001999912], [-99.09507622999995, 19.209741369999165], [-99.09506453999994, 19.210078979999125], [-99.094996, 19.21107427999916], [-99.09497577999996, 19.21115116999909], [-99.09494545999993, 19.211220849999158], [-99.09489991999997, 19.211283319999147], [-99.09483414999991, 19.211360799999085], [-99.0947430799999, 19.21143290999914], [-99.09465450999991, 19.2114930199991], [-99.09456087999993, 19.21153870999915], [-99.09448494999994, 19.211553149999084], [-99.09439634999991, 19.21155319999908], [-99.0942950799999, 19.211524429999145], [-99.0942043099999, 19.21147402999912], [-99.09400930999998, 19.2113083699991], [-99.09368524999994, 19.211078919999103], [-99.09354214999992, 19.211008709999135], [-99.0933889199999, 19.210964489999128], [-99.09318091999992, 19.210929939999076], [-99.09292934999993, 19.21091876999912], [-99.09239171999994, 19.210892509999177], [-99.09171704999993, 19.210920089999135], [-99.09096536999998, 19.211019379999136], [-99.0908728799999, 19.21104822999913], [-99.0906947999999, 19.211095649999134], [-99.08903254999993, 19.211436579999134], [-99.08850425999995, 19.21154494999917], [-99.08822804999991, 19.21160159999912], [-99.08631317999995, 19.2119943199991], [-99.08440964999993, 19.212384709999117], [-99.08377008999996, 19.21251585999914], [-99.08305105999993, 19.212663309999105], [-99.08250902999997, 19.21277697999912], [-99.08196607999996, 19.212890919999115], [-99.0813517999999, 19.213019819999147], [-99.08089568999993, 19.213115519999114], [-99.08035562999994, 19.213227649999126], [-99.08005873999993, 19.213289279999113], [-99.07937191999993, 19.21343185999914], [-99.0788344599999, 19.213543449999115], [-99.07870404999991, 19.213570519999163], [-99.07853377, 19.213605879999136], [-99.07812794999994, 19.213690109999188], [-99.07781996999992, 19.213754049999118], [-99.07772102999992, 19.21377457999914], [-99.0767535499999, 19.213975429999145], [-99.0757902299999, 19.214175379999137], [-99.07358662999995, 19.21463277999911], [-99.0723071799999, 19.21489834999912], [-99.0710643099999, 19.21515630999912], [-99.0690714899999, 19.21556988999915], [-99.0677717999999, 19.215839599999104], [-99.0673254599999, 19.215932229999165], [-99.06630867999992, 19.216143229999158], [-99.06580381999996, 19.216247979999125], [-99.06433019999994, 19.216553769999177], [-99.06320779999994, 19.21678664999913], [-99.06054339999993, 19.217338429999092], [-99.06015719999995, 19.217418419999134], [-99.0591893799999, 19.21761882999914], [-99.05816330999993, 19.217831309999152], [-99.05671625999994, 19.218130939999128], [-99.05635317999993, 19.21820611999913], [-99.05598373999996, 19.218282619999115], [-99.05383919999993, 19.21872764999916], [-99.0530976, 19.21888153999912], [-99.05235468999992, 19.219035689999146], [-99.04914367999994, 19.219701939999137], [-99.04826553999995, 19.21988411999915], [-99.04786719999994, 19.219966769999107], [-99.046644, 19.220220539999183], [-99.04543821999992, 19.220470709999088], [-99.04302259999994, 19.22097185999915], [-99.04278534999993, 19.22102107999914], [-99.04246805999995, 19.22108690999913], [-99.04216881999996, 19.221149029999125], [-99.04178645999993, 19.221228399999113], [-99.04112473999996, 19.221365749999148], [-99.0401053099999, 19.221577349999134], [-99.0380401399999, 19.2220059999991], [-99.03759659999993, 19.222098049999147], [-99.03693965999993, 19.222234399999138], [-99.03673162999995, 19.22227756999914], [-99.03641890999992, 19.222342399999132], [-99.03628364999996, 19.22237042999915], [-99.03578639999994, 19.22247350999917], [-99.0352087699999, 19.222593249999086], [-99.03406104999998, 19.22283114999913], [-99.03287417999994, 19.22307714999914], [-99.0301930299999, 19.223632849999145], [-99.02627388999993, 19.223980769999095], [-99.02532771999994, 19.22406464999913], [-99.025018, 19.224092109999166], [-99.02300388999996, 19.224270629999133], [-99.02271196999995, 19.224296509999093], [-99.02173831999993, 19.224382849999124], [-99.02119048999994, 19.224431429999125], [-99.01998742999993, 19.22453810999912], [-99.01962417999992, 19.224570319999142], [-99.0182038199999, 19.224696249999106], [-99.01778959999992, 19.224732969999156], [-99.0174833199999, 19.22476024999913], [-99.01616273999996, 19.224877859999108], [-99.01588016999993, 19.224903019999097], [-99.01521133999992, 19.224962569999118], [-99.01510928999991, 19.2249716599991], [-99.01315771999997, 19.22514541999914], [-99.01267353999994, 19.225188519999154], [-99.01249421999991, 19.225204489999136], [-99.01206013999995, 19.22524313999913], [-99.01160439999994, 19.22528370999912], [-99.01044628999993, 19.225386779999166], [-99.00889496999993, 19.22552485999915], [-99.00852974999994, 19.225558089999126], [-99.00755936999995, 19.225646339999077], [-99.0071519699999, 19.225683379999165], [-99.00580005999994, 19.225806319999133], [-99.00573873999993, 19.225811889999118], [-99.00489611999991, 19.225888509999127], [-99.00444207999995, 19.22592979999919], [-99.0040548599999, 19.225964999999082], [-99.00397048999996, 19.225972679999117], [-99.00301901999991, 19.226059169999136], [-99.0021401399999, 19.226139079999164], [-99.00166196999994, 19.226182539999126], [-99.00148781999995, 19.226198369999153], [-99.0001037499999, 19.22631757999917], [-99.00009611999995, 19.226392829999156], [-99.003129, 19.227060019999133], [-99.00299064999994, 19.227573919999152], [-99.0022150899999, 19.227357429999135], [-99.0007657399999, 19.22699437999916], [-98.99984690999995, 19.226783919999143], [-98.9996738299999, 19.227282689999118], [-98.99957247999996, 19.22755950999916], [-98.99902228999994, 19.227357049999085], [-98.99886393999994, 19.227704739999123], [-98.99814737999993, 19.2275319099991], [-98.99717701999995, 19.22747256999914], [-98.99723373999994, 19.22731068999911]]]}",
#                "municipio": 9
#            }

In [10]:
#Migrate the models to persist into the database
TransportUnit.__table__.create(engine)
Mayoralty.__table__.create(engine)

In [11]:
#Request logic, request, transform and load informations from the alcaldias endpoint to store in the database as geom data
url_alcaldias = "https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=e4a9b05f-c480-45fb-a62c-6d4e39c5180e&limit=50"
response = requests.get(url_alcaldias)
#Parse the response to json format
json_response = response.json()
#Validate if there are records in the response
if json_response != {}:
    #Retrieve only the result 
    records = json_response['result']['records']
    #Iterate the parsed response
    for record in records:
        #Apply the transformation function
        clean_record = transform_to_list(record)
         #Generate a new instance of the model
        mayoralty = Mayoralty(clean_record)
        session.add(mayoralty)
#Persist the iteration
session.commit()

In [12]:
#Request logic, request, transform and load informations from the metrobus endpoint to store in the database as geom data
url = "https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=ad360a0e-b42f-482c-af12-1fd72140032e&limit=200"
response = requests.get(url)
#Parse the response to json format
json_response = response.json()
#print(json_response)
#Validate if there are records in the response
if json_response != {}:
    #Retrieve only the records
    records = json_response['result']['records']
    #Iterate the parsed response
    for record in records:
        #Apply the transformation function
        clean_record = transform_to_list(record)
        #Generate a new instance of the model
        unit = TransportUnit(clean_record)
        #Retrieve the point to query using the GeORM
        point = unit.geographic_point
        #Verify if the point indicated on the geom column belongs to any mayoralty
        mayoralty = session.query(Mayoralty).filter(Mayoralty.geo_shape.ST_Contains(point)).first()
        #If ther's a matching mayoralty persist in to the transport unit column
        if mayoralty != None:
            unit.mayoralty = mayoralty.nomgeo
        session.add(unit)
#Persist the iteration
session.commit()



hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola


In [15]:
#Locate the mayoralty that the transport unit is in 
alcaldias = session.query(Mayoralty).filter(Mayoralty.nomgeo == "Milpa Alta").all()